In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [4]:
movie_embeddings_matrix = np.load('../data/vector_db/movie_embeddings_matrix.npy')
user_embeddings_matrix = np.load('../data/vector_db/user_embeddings_matrix.npy')
user2Idx = np.load('../data/vector_db/user2Idx.npy', allow_pickle=True).item()
movie2Idx = np.load('../data/vector_db/movie2Idx.npy', allow_pickle=True).item()

In [24]:
df_users = pd.read_csv('../data/usuarios.csv')
df_movies = pd.read_csv('../data/peliculas.csv')
ratings = pd.read_csv('../data/scores.csv')
df_movies.loc[df_movies['IMDB URL'].isna(), 'IMDB URL'] = ''

user_movies = ratings[ratings['user_id'] == 196]
user_movies = user_movies.loc[user_movies['rating'].idxmax()]
user_movies

id                         1896
user_id                     196
movie_id                    655
rating                        5
Date        1997-12-04 16:09:53
Name: 1896, dtype: object

In [7]:
df_users['userIdx'] = df_users['id'].apply(lambda x: user2Idx[x])
df_movies['movieIdx'] = df_movies['id'].apply(lambda x: movie2Idx[x])

,id,Occupation,Active Since,userIdx
0,1,technician,1997-09-22 21:57:58,119
1,2,other,1998-02-27 03:26:00,230
2,3,writer,1998-03-07 02:15:39,270
3,4,technician,1998-04-08 02:09:34,268
4,5,other,1997-09-30 15:57:44,112
...,...,...,...,...
938,939,student,1997-11-23 04:50:36,939
939,940,administrator,1998-01-14 18:01:38,932
940,941,student,1997-09-23 21:01:35,943
941,942,librarian,1998-03-30 18:25:55,935


In [8]:
import sys
import os
notebook_dir = os.path.dirname(os.path.abspath('recommendation_system.ipynb'))
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(project_root)

#from config.config import client
from src.services.database.models import Movie, User
from src.services.database.vector_db_service import VectorDbService
vdbs = VectorDbService()
client = vdbs.client
client.cluster.health()

{'cluster_name': 'docker-cluster',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'discovered_master': True,
 'discovered_cluster_manager': True,
 'active_primary_shards': 13,
 'active_shards': 13,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 9,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 59.09090909090909}

In [8]:
try:
    #client.indices.delete('movie')
    Movie.init(using=client)
    print("Índice inicializado correctamente.")
except Exception as e:
    print(f"Error al inicializar el índice: {e}")

Índice inicializado correctamente.


In [14]:
try:
    #client.indices.delete('user')
    User.init(using=client)
    print("Índice inicializado correctamente.")
except Exception as e:
    print(f"Error al inicializar el índice: {e}")

Índice inicializado correctamente.


In [10]:
for i, row in df_movies.iterrows():
    try:

        mv = Movie(
            movie_id = row.id,
            url = row['IMDB URL'],
            name = row['Name'],
            vector = list(movie_embeddings_matrix[row.movieIdx]),
            created_at = datetime.now()
        )
        mv.save(using=client)
        #Movie.search(using=client).count()
        #mv.save(using=client)
    except Exception as e:
        print(f"Error al guardar la película {row['Name']}: {e}")

In [15]:
for i, row in df_users.iterrows():
    try:

        uv = User(
            user_id = row.id,
            occupation = row['Occupation'],
            created_at = datetime.now(),
            active_since = row['Active Since'],
            vector = list(user_embeddings_matrix[row.userIdx])
        )
        uv.save(using=client)
    except Exception as e:
        print(f"Error al guardar el usuario {row.id}: {e}")

In [11]:
Movie.search(using=client).count()

1682

In [16]:
User.search(using=client).count()

943

In [12]:
movie_idx_to_search = 5

df_movies[df_movies['movieIdx'] == movie_idx_to_search]
# %%
movie_embeddings_matrix[movie_idx_to_search]

query = {
    "size": 5,
    "query": {
        "knn": {
        "vector": {
            "vector": movie_embeddings_matrix[movie_idx_to_search],
            "k" : 20
        }
        }
    }
}

response = client.search(index='movie', body=query)

for h in response['hits']['hits']:
  print(h)

{'_index': 'movie', '_id': '346', '_score': 1.0, '_source': {'movie_id': 346, 'url': 'http://us.imdb.com/M/title-exact?imdb-title-119396', 'name': 'Jackie Brown (1997)', 'vector': [0.0675366222858429, 0.09568929672241211, 0.22328245639801025], 'created_at': '2024-08-28T18:39:01.838179'}}
{'_index': 'movie', '_id': '987', '_score': 0.99915195, '_source': {'movie_id': 987, 'url': 'http://us.imdb.com/M/title-exact?Underworld%20(1997)', 'name': 'Underworld (1997)', 'vector': [0.000920130405575037, 0.0010935215977951884, 0.0028165383264422417], 'created_at': '2024-08-28T18:39:05.507450'}}
{'_index': 'movie', '_id': '285', '_score': 0.9991291, '_source': {'movie_id': 285, 'url': 'http://us.imdb.com/M/title-exact?Secrets%20&%20Lies%20(1996)', 'name': 'Secrets & Lies (1996)', 'vector': [0.06075748801231384, 0.08659423142671585, 0.22390972077846527], 'created_at': '2024-08-28T18:39:01.557003'}}
{'_index': 'movie', '_id': '1411', '_score': 0.99898773, '_source': {'movie_id': 1411, 'url': 'http:/